**Reference: [Deep Residual Learning for Image Recognition](https://arxiv.org/abs/1512.03385)**

Adapted from code contributed by **[donnemartin](https://github.com/donnemartin/data-science-ipython-notebooks/tree/master/deep-learning/keras-tutorial/deep_learning_models)**

In [2]:
import numpy as np
import warnings
from keras.layers import Input
from keras.layers import add
from keras.layers import Dense, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D, AveragePooling2D
from keras.layers import BatchNormalization
from keras.models import Model
from keras.preprocessing import image
import keras.backend as K
from keras.utils.data_utils import get_file

/Users/quansun/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
WEIGHTS_PATH = 'https://github.com/fchollet/deep-learning-models/releases/download/v0.2/resnet50_weights_tf_dim_ordering_tf_kernels.h5'
WEIGHTS_PATH_NO_TOP = 'https://github.com/fchollet/deep-learning-models/releases/download/v0.2/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5'

In [4]:
def identity_block(input_tensor,kernel_size,filters,stage,block):
    nb_filter1, nb_filter2, nb_filter3 = filters
    bn_axis=3
    
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    x = Convolution2D(nb_filter1, (1, 1), name=conv_name_base + '2a')(input_tensor)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2a')(x)
    x = Activation('relu')(x)
    
    x = Convolution2D(nb_filter2,kernel_size=kernel_size,
                      padding='same',name=conv_name_base+'2b')(x)
    x = BatchNormalization(axis=bn_axis,name=bn_name_base+'2b')(x)
    x = Activation('relu')(x)
    
    x = Convolution2D(nb_filter3, (1, 1), name=conv_name_base+'2c')(x)
    x = BatchNormalization(axis=bn_axis,name=bn_name_base+'2c')(x)
    
    x = add([x, input_tensor])
    x = Activation('relu')(x)
    return x

In [5]:
def conv_block(input_tensor,kernel_size,filters,stage,block,strides=(2,2)):
    nb_filter1,nb_filter2,nb_filter3 = filters
    if K.image_dim_ordering() == 'tf':
        bn_axis=3
    else:
        bn_axis=1
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'res' + str(stage) + block + '_brach'
    
    x = Convolution2D(nb_filter1,(1,1),strides=strides,
                     name=conv_name_base+'2a')(input_tensor)
    x = BatchNormalization(axis=bn_axis,name=bn_name_base+'2a')(x)
    x = Activation('relu')(x)
    
    x = Convolution2D(nb_filter2,kernel_size,
                     padding='same',name=conv_name_base+'2b')(x)
    x = BatchNormalization(axis=bn_axis,name=bn_name_base+'2b')(x)
    x = Activation('relu')(x)
    
    x = Convolution2D(nb_filter3,(1,1),name=conv_name_base+'2c')(x)
    x = BatchNormalization(axis=bn_axis,name=bn_name_base+'2c')(x)
    
    shortcut = Convolution2D(nb_filter3,(1,1),strides=strides,
                           name=conv_name_base+'1')(input_tensor)
    shortcut = BatchNormalization(axis=bn_axis,name=bn_name_base+'1')(shortcut)
    
    x = add([x, shortcut])
    x = Activation('relu')(x)
    return x

In [6]:
def ResNet50(include_top=True,weights='imagenet',
            input_tensor=None):
    ''' Arguments
        include_top: whether to include the 3 fully-connected
            layers at the top of the network.
        weights: one of `None` (random initialization)
            or "imagenet" (pre-training on ImageNet).
        input_tensor: optional Keras tensor'''
    if weights not in {'imagenet',None}:
        raise ValueError("The 'weights' argument should be either 'None' (random initialization) or 'imagenet' (pre-training on ImageNet)")
    if include_top:
            input_shape=(224,224,3)
    else:
            input_size=(None,None,3)
    if input_tensor is None:
        img_input = Input(shape=input_shape)
    else:
        if not K.is_keras_tensor(input_tensor):
            img_input = Input(tensor=input_tensor)
        else:
            img_input = input_tensor
    
    bn_axis=3
        
    x = ZeroPadding2D((3,3))(img_input)
    x = Convolution2D(64,(7,7),strides=(2,2),name='conv1')(x)
    x = BatchNormalization(axis=bn_axis,name='bn_conv1')(x)
    x = Activation('relu')(x)
    x = MaxPooling2D((3,3),strides=(2,2))(x)
    
    x = conv_block(x,(3,3),[64,64,256],stage=2,block='a',strides=(1,1))
    x = identity_block(x,(3,3),[64,64,256],stage=2,block='b')
    x = identity_block(x,(3,3),[64,64,256],stage=2,block='c')
    
    x = conv_block(x,(3,3),[128,128,512],stage=3,block='a')
    x = identity_block(x,(3,3),[128,128,512],stage=3,block='b')
    x = identity_block(x,(3,3),[128,128,512],stage=3,block='c')
    x = identity_block(x,(3,3),[128,128,512],stage=3,block='d')
    
    x = conv_block(x,(3,3),[256,256,1024],stage=4,block='a')
    x = identity_block(x,(3,3),[256,256,1024],stage=4,block='b')
    x = identity_block(x,(3,3),[256,256,1024],stage=4,block='c')
    x = identity_block(x,(3,3),[256,256,1024],stage=4,block='d')
    x = identity_block(x,(3,3),[256,256,1024],stage=4,block='e')
    x = identity_block(x,(3,3),[256,256,1024],stage=4,block='f')
    
    x = conv_block(x,(3,3),[512,512,2048],stage=5,block='a')
    x = identity_block(x,(3,3),[512,512,2048],stage=5,block='b')
    x = identity_block(x,(3,3),[512,512,2048],stage=5,block='c')
    
    x = AveragePooling2D((7,7),name='avg_pool')(x)
    
    if include_top:
        x = Flatten()(x)
        x = Dense(1000,activation='softmax',name='fc1000')(x)
    
    model = Model(img_input,x)
    if weights=='imagenet':
        if include_top:
                weights_path = get_file('resnet50_weights_tf_dim_ordering_tf_kernels.h5',
                                        WEIGHTS_PATH,
                                        cache_subdir='models',
                                        md5_hash='a7b3fe01876f51b976af0dea6bc144eb')
        else:
            weights_path = get_file('resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5',
                                    WEIGHTS_PATH_NO_TOP,
                                    cache_subdir='models',
                                    md5_hash='a268eb855778b3df3c7506639542a6af')
        model.load_weights(weights_path)
    return model

In [7]:
def preprocess_input(x):
    x[:, :, :, 0] -= 103.939
    x[:, :, :, 1] -= 116.779
    x[:, :, :, 2] -= 123.68
    # 'RGB'->'BGR'
    x = x[:, :, :, ::-1]
    return x

In [8]:
import json
CLASS_INDEX = None
CLASS_INDEX_PATH = 'https://s3.amazonaws.com/deep-learning-models/image-models/imagenet_class_index.json'
def decode_predictions(preds):
    global CLASS_INDEX
    assert len(preds.shape) == 2 and preds.shape[1] == 1000
    if CLASS_INDEX is None:
        fpath = get_file('imagenet_class_index.json',
                         CLASS_INDEX_PATH,
                         cache_subdir='models')
        CLASS_INDEX = json.load(open(fpath))
    indices = np.argmax(preds, axis=-1)
    results = []
    for i in indices:
        results.append(CLASS_INDEX[str(i)])
    return results

In [9]:
model = ResNet50(include_top=True,weights='imagenet')
img_path='strawberry.jpeg'
img = image.load_img(img_path,target_size=(224,224))
x = image.img_to_array(img)
x = np.expand_dims(x,axis=0)
x = preprocess_input(x)
print ('Input image shape:',x.shape)

preds = model.predict(x)
print('Predicted:', decode_predictions(preds))

Input image shape: (1, 224, 224, 3)
Predicted: [['n07745940', 'strawberry']]
